# Recomendación. Campaña de email marketing.

# Objetivo

- Desarrollar una estrategia de email marketing dirigida a 10,000 clientes, priorizando los productos que generen mayor beneficio.



# Aspectos a tener en cuenta

1. Modelos de Predicción:

- Crear modelos predictivos para determinar qué productos tienen mayor probabilidad de ser contratados por cada cliente.
- Considerar no solo la probabilidad de contratación, sino también el beneficio económico de cada producto.
- Utilizar técnicas de modelado avanzadas como árboles de decisión, regresión logística y redes neuronales.
- Validar y evaluar los modelos utilizando métricas como la precisión, el recall y el AUC-ROC.
2. Selección de Clientes:
- Seleccionar los 10,000 clientes que recibirán los emails basados en los resultados del modelo predictivo.
- Justificar la elección de estos clientes y la estrategia empleada.

# Enfoques para la creación de los modelos predictivos

1. Creación de un modelo para cada producto:

- En este enfoque, se crearía un modelo predictivo separado para cada producto (por ejemplo, uno para short_term_deposit, otro para loans, otro para mortgage, etc.).

- Ventajas: Es específico para cada producto. El modelo se entrenaría específicamente para un producto, lo que permite que aprenda patrones únicos relacionados con la contratación de ese producto.
- Interpretabilidad: Los factores que influyen en la contratación de cada producto pueden ser más fáciles de interpretar.
- Desventajas: Sería costoso en términos de recursos ya que requiere entrenar y mantener múltiples modelos. Además, muchos de los modelos podrían utilizar características similares, lo que significa que se estaría trabajando doble.

2. Modelo agrupado por tipos de productos:

- Se agruparían los productos similares en categorías, exactamente como lo explicaba Erin (Responsable de Marketing Directo) en uno de sus correos [cuentas, productos de ahorro e inversión (planes, fondos, etc.) y financiación (préstamos y tarjetas)].

- Ventajas: Menos modelos que mantener en comparación con la opción de un modelo por producto. Además, se pueden identificar patrones que se aplican a categorías de productos.
- Desventajas: Puede que no capture detalles específicos de un solo producto tan bien como un modelo dedicado.

3. Modelo General para Todos los Productos:

- Se utilizaría un único modelo para predecir la probabilidad de que un cliente contrate cualquier producto, saliendo con una lista de productos con probabilidades.
- Ventajas: Es simple, solo hay que entrenar y mantener un único modelo, por tanto menos recursos computacionales y menos tiempo de entrenamiento.
- Desventajas: Puede ser más complejo interpretar las relaciones ya que el modelo tiene que captar información de todos los productos y si un producto es raro comparado con otros, el modelo puede no predecirlo correctamente.

Recomendación de Enfoque

Basado en los productos y tu conjunto de datos, podrías optar por un enfoque híbrido:

	1.	Empezar con un Modelo General: Entrenar un modelo inicial que prediga la probabilidad de contratación de productos en general. Este modelo puede ayudarte a identificar qué características son las más importantes para la contratación de productos en general.
	2.	Refinar con Modelos Específicos para Productos Clave: Identifica productos clave (por ejemplo, aquellos que son más rentables o tienen un interés estratégico particular). Entrena modelos específicos para estos productos, utilizando las características identificadas en el modelo general.
	3.	Agrupación por Categorías: Para productos que son similares o están relacionados, considera agruparlos en categorías y entrenar modelos basados en estas categorías.

Proceso para Entrenar los Modelos

Independientemente del enfoque elegido, los siguientes pasos te ayudarán a entrenar y evaluar tus modelos:

	1.	Preprocesamiento de Datos:
	•	Codificación de Variables Categóricas: Usar técnicas como codificación one-hot o codificación de etiquetas para columnas categóricas como region_code, gender, entry_channel, etc.
	•	Escalado de Características: Escalar características numéricas (como age, salary) para estandarizar el rango de valores.
	•	Manejo de Valores Faltantes: Decidir cómo tratar con valores faltantes (imputación, eliminación, etc.).
	2.	División del Conjunto de Datos:
	•	Dividir los datos en conjuntos de entrenamiento y prueba. Usualmente se usa una proporción de 80/20 o 70/30.
	•	Opcional: Dividir en conjunto de entrenamiento, validación y prueba para ajuste de hiperparámetros.
	3.	Selección de Modelos:
	•	Utilizar algoritmos de clasificación como regresión logística, árboles de decisión, bosques aleatorios, XGBoost, o redes neuronales dependiendo de la complejidad del problema y los datos disponibles.
	4.	Entrenamiento y Evaluación:
	•	Entrenar el modelo usando el conjunto de entrenamiento.
	•	Evaluar el modelo usando métricas como precisión, recall, F1-score, y ROC-AUC para clasificar los resultados.
	•	Ajustar hiperparámetros utilizando técnicas como grid search o random search.
	5.	Predicción y Recomendación:
	•	Utilizar los modelos para predecir las probabilidades de éxito de productos específicos para cada cliente.
	•	Priorizar las recomendaciones basadas en las probabilidades predichas.

Ejemplo de Implementación de un Modelo General en Python

A continuación, se muestra un ejemplo básico utilizando un modelo de clasificación para predecir la probabilidad de contratación de un producto genérico:

In [ ]:
# cargando el dataset de productos desde S3 de AWS


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report


ca_df = pd.read_parquet('https://easy-money-project-bucket.s3.eu-west-3.amazonaws.com/products_df.parquet')
# Suponiendo que ca_df es tu DataFrame con los datos históricos
# Lista de productos
productos = ['credit_card', 'loans', 'mortgage', 'short_term_deposit']

# Diccionario para almacenar modelos y beneficios
modelos = {}
beneficios_por_producto = {
    'credit_card': 50,   # Beneficio estimado en dólares
    'loans': 500,
    'mortgage': 1000,
    'short_term_deposit': 100
}

# Para cada producto, crear y entrenar un modelo
for producto in productos:
    # Crear una columna de target para cada producto
    ca_df[f'target_{producto}'] = ca_df[producto]
    
    # Definir las características y la variable target
    X = ca_df.drop([f'target_{producto}', producto, 'pk_partition', 'Unnamed: 0'], axis=1)
    X.fillna(0, inplace=True)
    y = ca_df[f'target_{producto}']
    
    # Convertir variables categóricas a numéricas
    X = pd.get_dummies(X)
    
    # Dividir en conjunto de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Entrenar un modelo, por ejemplo, un RandomForestClassifier
    model = RandomForestClassifier(n_estimators=5, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    
    # Guardar el modelo entrenado
    modelos[producto] = model
    
    # Evaluar el modelo
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]
    print(f"Evaluación del modelo para {producto}:")
    print(classification_report(y_test, y_pred))
    print(f"ROC-AUC Score: {roc_auc_score(y_test, y_pred_prob):.2f}")
    
    # Calcular probabilidades de contratación para todos los clientes en el conjunto original
    ca_df[f'probabilidad_{producto}'] = model.predict_proba(X)[:, 1]

    # Calcular el beneficio esperado para cada producto y cliente
    ca_df[f'beneficio_esperado_{producto}'] = ca_df[f'probabilidad_{producto}'] * beneficios_por_producto[producto]

# Sumar el beneficio esperado para todos los productos
ca_df['beneficio_total_esperado'] = ca_df[[f'beneficio_esperado_{producto}' for producto in productos]].sum(axis=1)

# Seleccionar los 10,000 clientes con mayor beneficio total esperado
top_10000_clientes = ca_df.sort_values('beneficio_total_esperado', ascending=False).head(10000)

# Mostrar los 10,000 clientes seleccionados
print(top_10000_clientes[['pk_cid', 'beneficio_total_esperado']])

/Users/claudiacastro/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Evaluación del modelo para credit_card:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1178265
           1       0.78      0.75      0.77     14320

    accuracy                           0.99   1192585
   macro avg       0.89      0.87      0.88   1192585
weighted avg       0.99      0.99      0.99   1192585

ROC-AUC Score: 0.96
Evaluación del modelo para loans:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1192490
           1       0.93      0.84      0.88        95

    accuracy                           1.00   1192585
   macro avg       0.97      0.92      0.94   1192585
weighted avg       1.00      1.00      1.00   1192585

ROC-AUC Score: 0.97
Evaluación del modelo para mortgage:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1192519
           1       0.98      0.95      0.97        66

    accuracy                   

In [14]:
0.95**2

0.9025

In [16]:
0.5**3

0.125

In [17]:
0.5**3*40

5.0

In [ ]:
#### Comportamientos clientes que se repiten en las particiones

In [ ]:
# Crear la tabla de presencia
presence_table = pd.crosstab(ca_df['pk_cid'], ca_df['pk_partition'])

# Visualizar la tabla de presencia
presence_table.reset_index(inplace=True)
presence_table

In [ ]:
# Crear un mapa de calor de actividad de clientes
plt.figure(figsize=(12, 8))
sns.heatmap(presence_table.drop('pk_cid', axis=1), cmap='YlGnBu', cbar=True)
plt.xlabel('Mes')
plt.ylabel('Cliente (pk_cid)')
plt.title('Mapa de Calor de la Actividad de Clientes por Mes')
plt.show()